In [1]:
from __future__ import print_function

import os
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from architecture import MST
torch.cuda.manual_seed(seed=666)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)


########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
YY= mat73.loadmat("./Data/T31UDQ/T31UDQ_20m_raw1.mat")['T31UDQ_20m_raw1'] 
YY = np.double(YY)
YY = YY/YY.max()
Mask1_500 = mat73.loadmat("./Data/Mask1_500.mat")['Mask1_500']
Mask2_500 = mat73.loadmat("./Data/Mask2_500.mat")['Mask2_500']

Mask = np.copy(Mask2_500)
Mask[ Mask != 0 ] = 1
# Test1_Clean是干净图像，取的是4，2，0三个时相
Test1_Clean = np.copy(YY[:, :, :, [4, 2, 0]])

# 生成云图YY，含有6个时相
Clean = YY[:, :, :, [4, 2, 0, 5, 3, 1]]
YY = np.multiply((1 - Mask), (YY[:, :, :, [4, 2, 0, 5, 3, 1]])) + Mask
Noisy = np.copy(YY)
Cloud = np.copy(Mask)
Mask = 1 - Cloud
###########################################################################

M, N, B, T = Noisy.shape
Clean = torch.from_numpy(Clean.reshape(M, N, B, T)).to(device)
Mask = torch.from_numpy(Mask.reshape(M, N, B, T)).to(device)  # 云=0
Cloud = torch.from_numpy(Cloud.reshape(M, N, B, T)).to(device)  # 云=1
Noisy = torch.from_numpy(Noisy.reshape(M, N, B, T)).to(device)


################################ 全局设置 ################################
def grid_combine(args_list):
    for rate in args_list[0]:
        for lambda2 in args_list[1]:
            for rho in args_list[2]:
                for alpha in args_list[3]:
                    if rate == 125 and rho ==0.08 and alpha==1.03 :
                        continue
                    yield rate, lambda2, rho, alpha
args_list = [
    [125, 100],  # rate
    [0.1],  # lambda2
    [0.08, 0.06, 0.04],  # rho
    [1.03, 1.025]  # alpha
]
condidates = [
    # lambda1, lambda2, rho
    # [6.0, 0.1, 0.12, 1.03]     # condidate2
    # [12.5, 0.1, 0.1], 
    # [10.0, 0.1, 0.1], 
    # [7.50, 0.1, 0.1],  
    # [10.0, 0.2, 0.1], 
    # [10.0, 0.4, 0.1], 
]
condidates = [
    # rate, lambda2, rho, alpha
    # [90, 0.06, 0.06, 1.03],
    # [95, 0.06, 0.08, 1.03],
    # [88, 0.06, 0.08, 1.03],
    # [90, 0.08, 0.08, 1.03],
    [30, 0.4, 0.06, 1.0],
    [60, 0.4, 0.06, 1.02],
    [60, 0.2, 0.06, 1.03],
    [50, 0.4, 0.06, 1.03],
]
iter_num, epoch_num = 150, 300
order = 4
for rate, lambda2, rho, alpha in condidates:
# for rate, lambda2, rho, alpha in grid_combine(args_list):
    # rate = lambda1 / rho
    lambda1 = rate * rho
    iters = 0
    order += 1
    log_dir = "./log-candidates-picardie-acc/order[%03d]-iter[%3d]-epoch[%3d]-rate[%.3f]-lambda2[%.3f]-rho[%.3f,%.3f]" \
              % (order, iter_num, epoch_num, rate, lambda2, rho, alpha)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    else:
        continue
    # rate, lambda2, rho, alpha = 125, 0.1, 0.08, 1.04
    ################################ 初始化 ################################
    Y = Noisy.clone()
    X = Noisy.clone()
    W = Noisy.clone()
    C = torch.zeros(Y.shape).type(dtype).to(device)
    M_updata = Mask.clone()
    model = MST(dim=B*T, stage=2, num_blocks=[2, 2, 2]).cuda()
    psnr = 0
    s0_min = 10000
    ################################ 子问题更新 ################################
    while iters < iter_num:
        # if iters >= 100 and psnr < 32:
        #     break
        iters += 1
        ################################ 更新子问题X ################################
        # if decrease and (iters % 4 == 0) and LR > 0.00001:
        #     LR *= 0.9
        LR = 0.0002
        optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
        # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [250], gamma=0.7, last_epoch=-1)
        loss_history, loss1_history, loss2_history = [], [], []
        epoch = 0
        net_input = torch.permute(X, [2, 3, 0, 1]).reshape(1, B * T, M, N).type(dtype).to(device)
        while epoch < epoch_num:
            epoch += 1
            optimizer.zero_grad()
            # X: M, N, B, T
            Mask_temp = torch.permute(M_updata, [2, 3, 1, 0]).reshape(1, B*T, M, N).type(dtype).to(device)
            out = model(net_input, Mask_temp)
            X = torch.permute(out.reshape(B, T, M, N), [2, 3, 0, 1])
            loss1 = 1 / 2 * torch.norm(Y - torch.mul(X, M_updata) - C, 'fro') ** 2
            loss2 = rho / 2 * torch.norm(X - W, 'fro') ** 2
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            # scheduler.step()
            loss_history.append(loss.item())
            loss1_history.append(loss1.item())
            loss2_history.append(loss2.item())
            if epoch % 40 == 0:
                print("\tThe %03dth iters, the %03dth epoch, loss: %.5f" % (iters, epoch, loss.item()))
            Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()

        Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()

        ################################ 更新子问题W ################################
        U, s, VH = torch.linalg.svd(X.reshape(M * N, B * T), full_matrices=False)  # B*T, M*N
        s0 = s[0]
        rho *= alpha
        # if s0<s0_min: # s0处于不断下降的第一阶段
        #     s0_min = s0
        #     rho *= alpha
        # else:
        #     rho *= 1.02
        rate = lambda1 / rho
        print("s: ", s)
        s = s - lambda1 / rho
        s[s < 0] = 0
        S = torch.diag(s)
        W = torch.mm(torch.mm(U, S), VH).reshape(M, N, B, T)
        ################################ 更新子问题C ################################
        L = Y - torch.mul(X, M_updata)
        for t in range(T):
            C[:, :, :, t] = thres_21(L[:, :, :, t], lambda2, M, N, B)

        ################################ 更新掩码M ################################
        # B_mean = torch.mean(Y-X, dim=2).reshape(M, N, 1, T).repeat((1, 1, B, 1))
        # M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) > 0.01] = 0   # M_update=1表示无云，M_update=0表示云
        # if iters > 5:
        #     M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) < 0.01] = 1

        ################################ 进行指标评估 ################################
        # mse = torch.sum((X - Clean) ** 2)
        psnr = []
        for t in range(3):
            psnr_t = 0
            for b in range(B):
                psnr_t += compare_psnr(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy(), data_range=1)
            psnr.append(psnr_t / B)
        ssim = []
        for t in range(3):
            ssim_b = 0
            for b in range(B):
                ssim_b += compare_ssim(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy())
            ssim.append(ssim_b / B)
        # print("The %03dth iters, the %03dth epoch, loss: %.5f, mse: %.5f, psnr: %.5f, ssim: %.5f" % (
        #     iters, epoch, loss.item(), mse, psnr, ssim))

        image_Clean = Clean.cpu().numpy()
        image_Y = Y.cpu().numpy()
        image_X = X.cpu().detach().numpy()
        image_C = C.cpu().numpy()
        image_M = M_updata.cpu().numpy()
        plt.figure(figsize=(20, 20))
        for i in range(6):
            plt.subplot(6, 6, 1 + i)
            plt.title("Clean")
            plt.imshow(image_Clean[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 7 + i)
            plt.title("X")
            plt.imshow(image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 13 + i)
            plt.title("Y")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 19 + i)
            plt.title("Y-X")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5 - image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 25 + i)
            plt.title("C")
            plt.imshow(image_C[:, :, 0, i]*5, cmap='gray')
            plt.axis('off')

            plt.subplot(6, 6, 31 + i)
            plt.title("M_update")
            plt.imshow(image_M[:, :, 0, i], cmap='gray')
            plt.axis('off')
        psnr1, psnr2, psnr3 = psnr
        ssim1, ssim2, ssim3 = ssim
        result_path = "%s/iter[%03d, %03d]-pnsr[%.3f, %.3f, %.3f]-ssim[%.3f, %.3f, %.3f].png" % (
            log_dir, iters, epoch, psnr1, psnr2, psnr3, ssim1, ssim2, ssim3)
        plt.savefig(result_path)
        plt.clf()
        plt.figure(figsize=(30, 60))
        plt.subplot(6, 1, 1)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history[50:]))], loss_history[50:])

        plt.subplot(6, 1, 2)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history[50:]))], loss1_history[50:])

        plt.subplot(6, 1, 3)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history[50:]))], loss2_history[50:])

        plt.subplot(6, 1, 4)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history))], loss_history)

        plt.subplot(6, 1, 5)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history))], loss1_history)

        plt.subplot(6, 1, 6)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history))], loss2_history)

        loss_path = "%s/iter[%03d, %03d]-LR[%f]-rate[%.3f]-s0[%.3f]-rho[%.3f]-loss[%.3f].png" % (
            log_dir, iters, epoch, LR, rate, s0, rho, loss.item())
        plt.savefig(loss_path)
        plt.clf()

        plt.close()
        scio.savemat("%s/recover_data.mat" % log_dir, {"recover_data": image_X})
    # decrease = True
    # up = False


	The 001th iters, the 040th epoch, loss: 579080.72959
	The 001th iters, the 080th epoch, loss: 578902.12239
	The 001th iters, the 120th epoch, loss: 578863.96776
	The 001th iters, the 160th epoch, loss: 578841.52983
	The 001th iters, the 200th epoch, loss: 578825.97086
	The 001th iters, the 240th epoch, loss: 578814.19725
	The 001th iters, the 280th epoch, loss: 578804.98069
s:  tensor([955.6096, 442.9492, 285.5799, 110.1117,  30.0621,  11.6035,   7.5657,
          6.9671,   6.6210,   6.1634,   4.9711,   4.2871,   3.6906,   3.1644,
          3.0652,   2.8293,   2.6992,   2.5851,   2.4375,   2.2623,   2.2294,
          2.1172,   2.0551,   1.9741,   1.9410,   1.8460,   1.8028,   1.7256,
          1.7040,   1.5801,   1.4856,   1.4360,   1.4220,   1.3968,   1.2240,
          1.1885], device='cuda:0')


d:\development\anaconda\envs\deep-todo\lib\site-packages\skimage\_shared\utils.py:348: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  return func(*args, **kwargs)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the va

	The 002th iters, the 040th epoch, loss: 15928.39175
	The 002th iters, the 080th epoch, loss: 15772.46434
	The 002th iters, the 120th epoch, loss: 15729.84496
	The 002th iters, the 160th epoch, loss: 15703.76571
	The 002th iters, the 200th epoch, loss: 15686.78532
	The 002th iters, the 240th epoch, loss: 15672.43275
	The 002th iters, the 280th epoch, loss: 15662.51461
s:  tensor([889.8168, 392.4908, 249.1708, 105.5134,  29.4859,  11.4686,   7.5481,
          7.0435,   6.7714,   6.1243,   5.0150,   4.1812,   3.6356,   3.2237,
          2.9607,   2.9237,   2.6896,   2.5879,   2.3608,   2.3464,   2.2324,
          2.1872,   2.1438,   2.0843,   2.0360,   1.9787,   1.9246,   1.8519,
          1.7770,   1.7165,   1.6604,   1.6399,   1.5608,   1.5427,   1.4402,
          1.3990], device='cuda:0')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

	The 003th iters, the 040th epoch, loss: 15741.87222
	The 003th iters, the 080th epoch, loss: 15729.36383
	The 003th iters, the 120th epoch, loss: 15723.28768
	The 003th iters, the 160th epoch, loss: 15717.64845
	The 003th iters, the 200th epoch, loss: 15712.08245
	The 003th iters, the 240th epoch, loss: 15706.46430
	The 003th iters, the 280th epoch, loss: 15700.71967
s:  tensor([824.5687, 344.4610, 214.4251, 103.3132,  29.0895,  11.3899,   7.6641,
          7.1471,   6.9253,   6.1603,   5.1056,   4.3032,   3.8608,   3.4450,
          3.2761,   3.0895,   2.9557,   2.8368,   2.7466,   2.5962,   2.5029,
          2.4722,   2.4425,   2.3622,   2.2915,   2.2309,   2.1255,   2.0631,
          2.0330,   1.9237,   1.9129,   1.8365,   1.7921,   1.7378,   1.6634,
          1.5969], device='cuda:0')


...

	The 074th iters, the 040th epoch, loss: 15603.27749
	The 074th iters, the 080th epoch, loss: 15603.18635
	The 074th iters, the 120th epoch, loss: 15603.15750
	The 074th iters, the 160th epoch, loss: 15603.12352
	The 074th iters, the 200th epoch, loss: 15603.08792
	The 074th iters, the 240th epoch, loss: 15603.04359
	The 074th iters, the 280th epoch, loss: 15602.95474
s:  tensor([212.3120,  27.8632,  11.4003,  10.3290,   8.8404,   6.5518,   6.3631,
          5.5793,   5.0903,   4.7335,   3.6010,   2.9446,   2.7525,   2.2151,
          1.7753,   1.5069,   1.4328,   1.4033,   1.2885,   1.2470,   1.2102,
          1.1201,   1.1163,   1.0610,   1.0168,   0.9818,   0.9535,   0.9378,
          0.9038,   0.8580,   0.8287,   0.7742,   0.7672,   0.7329,   0.7113,
          0.6767], device='cuda:0')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

	The 075th iters, the 040th epoch, loss: 15602.82962
	The 075th iters, the 080th epoch, loss: 15602.75734
	The 075th iters, the 120th epoch, loss: 15602.73329
	The 075th iters, the 160th epoch, loss: 15602.70812
	The 075th iters, the 200th epoch, loss: 15602.67729
	The 075th iters, the 240th epoch, loss: 15602.64150
	The 075th iters, the 280th epoch, loss: 15602.58558
s:  tensor([212.2906,  27.8152,  11.3808,  10.2793,   8.8224,   6.5510,   6.3549,
          5.5758,   5.0779,   4.7272,   3.5875,   2.9361,   2.7455,   2.2071,
          1.7608,   1.4886,   1.4143,   1.3893,   1.2730,   1.2282,   1.1980,
          1.1101,   1.1014,   1.0488,   1.0037,   0.9679,   0.9379,   0.9273,
          0.8876,   0.8445,   0.8160,   0.7643,   0.7551,   0.7203,   0.7017,
          0.6667], device='cuda:0')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

	The 076th iters, the 040th epoch, loss: 15602.91761


In [ ]:
_, a, _ = torch.linalg.svd(X.reshape(M*N, B*T), full_matrices=False)  # B*T, M*N
a

In [ ]:
psnr = []
for t in range(3):
    psnr_b = 0
    for b in range(B):
        psnr_b += compare_psnr(image_X[:, :, b, t], Clean[:, :, b, t].detach().cpu().numpy(), data_range=1)
    psnr_b /= B
    psnr.append(psnr_b)
print(psnr)

In [ ]:
scio.savemat("%s/recover_data.mat" % log_dir, {"recover_data": image_X})

In [ ]:
a = scio.loadmat("%s/recover_data.mat" % log_dir)["recover_data"]
